# Basic setup for fetching data from the facebook api

### Remember to add an active access token below and adjust the parameters and fields if necessary

In [2]:

# must be specified to be able to use methods of package
import sys
sys.path.append('../src') 
# import packages
from political_ads.helper import *
import pandas as pd
from political_ads.api_request import get_json_response

# insert your access token here (expires every 2 hours)
access_token = "EAADqFZC76BDMBAKhvS0dJ4wGOkX0mF14FZCKfF9bYTzIImcuXAS1AWu4X0XIwx0oN5bDpusy7heDf5CbfZBhPCatpLoFtlpslHHFjiRVmF031tKz0QsrOvOhZBMVMbw5f36OTeDTZBHMJJkDkDgZC5Y2TAjLHNUvt4DZCBjnzSIJFrJCzi2Co8myZC30kFkfSf9KdFRAmfB0aOgAAZBwZBbj4h"
# get_new_access_token: https://developers.facebook.com/tools/explorer/


# PARAMETERS NECESSARY
ad_type = "POLITICAL_AND_ISSUE_ADS"
ad_reached_countries = ['US'] # Facebook delivered the ads in these countries. Provided as ISO country codes.
search_terms = "climate change" # The terms to search for in your query. We treat a blank space as a logical AND and search for both terms and no other operators. The limit of your string is 100 characters or less.
limit = 1700
start_date = "2020-08-00"
end_date = "2020-11-03"


# FIELDS to specify your results
fields = ["id", "page_name", "ad_creative_body", "impressions", "delivery_by_region", "demographic_distribution", "ad_delivery_start_time", "bylines", "spend"]

# see here for more parameters and fields: https://www.facebook.com/ads/library/api/?source=archive-landing-page 



input_url = f"https://graph.facebook.com/v12.0/ads_archive?search_terms={search_terms}&ad_type={ad_type}&ad_reached_countries={ad_reached_countries}&fields={fields}&limit={limit}&ad_delivery_date_min={start_date}&ad_delivery_date_max={end_date}"

response = get_json_response(input_url, access_token)

data = pd.DataFrame(response["data"])


In [6]:
import time 

#https://developers.facebook.com/docs/graph-api/results

def larger_data_import(iterations, sleep_time):
    counter = 0
    for i in range(iterations):

        paging = response["paging"]["next"]
        next_response = get_json_response(paging, access_token)

        new_page = pd.DataFrame(next_response['data'])
        data = data.append(new_page, ignore_index=True)
        time.sleep(sleep_time)
        counter+=1
        print(counter)

#larger_data_import(10, 10)


In [7]:
#Write to CSV
#data.to_csv('test.csv')


In [8]:
#Cleaning the Data // Dropping missing values
data.dropna(inplace=True)

In [12]:

data.drop(['age_distribution', 'gender'], axis=1)

data['age_distribution'] = data['demographic_distribution'].apply(lambda x: get_age_distribution(x))

data['gender'] = data['demographic_distribution'].apply(lambda x: get_gender_distribution(x))

#data['impressions'] = data.impressions.apply(lambda x: transform_range(x))


In [ ]:
data

,id,page_name,ad_creative_body,impressions,delivery_by_region,demographic_distribution,ad_delivery_start_time,bylines,spend,new,gender,age_distribution
0,760713291157800,Boost the News,"Trump is a ""climate arsonist"" and ""climate den...","{'lower_bound': '0', 'upper_bound': '999'}","[{'percentage': '1', 'region': 'Wisconsin'}]","[{'percentage': '0.508772', 'age': '25-34', 'g...",2020-09-23,ACRONYM,"{'lower_bound': '0', 'upper_bound': '99'}",0.152047,0.152047,"{'25-34': 0.584795, '18-24': 0.333333999999999..."
1,653784258879707,Boost the News,"Trump is a ""climate arsonist"" and ""climate den...","{'lower_bound': '1000', 'upper_bound': '1999'}","[{'percentage': '1', 'region': 'Wisconsin'}]","[{'percentage': '0.001443', 'age': '18-24', 'g...",2020-09-23,ACRONYM,"{'lower_bound': '0', 'upper_bound': '99'}",0.167388,0.167388,"{'18-24': 0.258297, '35-44': 0.168831, '25-34'..."
2,621482761788913,Boost the News,"Trump is a ""climate arsonist"" and ""climate den...","{'lower_bound': '1000', 'upper_bound': '1999'}","[{'percentage': '1', 'region': 'Pennsylvania'}]","[{'percentage': '0.001205', 'age': '25-34', 'g...",2020-09-23,ACRONYM,"{'lower_bound': '0', 'upper_bound': '99'}",0.192772,0.192772,"{'25-34': 0.36626600000000004, '18-24': 0.5867..."
3,664254741166305,Boost the News,"Trump is a ""climate arsonist"" and ""climate den...","{'lower_bound': '4000', 'upper_bound': '4999'}","[{'percentage': '1', 'region': 'Pennsylvania'}]","[{'percentage': '0.003391', 'age': '25-34', 'g...",2020-09-23,ACRONYM,"{'lower_bound': '0', 'upper_bound': '99'}",0.232120,0.232120,"{'25-34': 0.290999, '35-44': 0.055795, '18-24'..."
5,358699412125930,Alliance for Climate Education,Why is the EPA letting coal-fired power plants...,"{'lower_bound': '0', 'upper_bound': '999'}","[{'percentage': '0.006757', 'region': 'Alabama...","[{'percentage': '0.027027', 'age': '35-44', 'g...",2020-11-03,Alliance for Climate Education,"{'lower_bound': '0', 'upper_bound': '99'}",0.425677,0.425677,"{'35-44': 0.040541, '65+': 0.70946, '55-64': 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1693,354761379087905,Future Coalition,There are millions of reasons to vote includin...,"{'lower_bound': '45000', 'upper_bound': '49999'}","[{'percentage': '0.000239', 'region': 'Hawaii'...","[{'percentage': '3.0E-5', 'age': '45-54', 'gen...",2020-10-23,Future Coalition,"{'lower_bound': '300', 'upper_bound': '399'}",0.335384,0.335384,"{'45-54': 0.009741999999999999, '18-24': 0.150..."
1696,385815939240103,Reverend Raphael Warnock,Please stop scrolling – this is REALLY importa...,"{'lower_bound': '0', 'upper_bound': '999'}","[{'percentage': '0.211009', 'region': 'Califor...","[{'percentage': '0.009174', 'age': '35-44', 'g...",2020-10-25,WARNOCK FOR GEORGIA,"{'lower_bound': '0', 'upper_bound': '99'}",0.467890,0.467890,"{'35-44': 0.3211, '65+': 0.12844, '45-54': 0.3..."
1697,671510653366602,Reverend Raphael Warnock,Please stop scrolling – this is REALLY importa...,"{'lower_bound': '0', 'upper_bound': '999'}","[{'percentage': '0.002667', 'region': 'Alabama...","[{'percentage': '0.005333', 'age': '55-64', 'g...",2020-10-25,WARNOCK FOR GEORGIA,"{'lower_bound': '0', 'upper_bound': '99'}",0.477334,0.477334,"{'55-64': 0.32, '65+': 0.22933300000000004, '3..."
1698,362482551766871,Reverend Raphael Warnock,Please stop scrolling – this is REALLY importa...,"{'lower_bound': '0', 'upper_bound': '999'}","[{'percentage': '0.002356', 'region': 'Alabama...","[{'percentage': '0.001178', 'age': '65+', 'gen...",2020-10-25,WARNOCK FOR GEORGIA,"{'lower_bound': '0', 'upper_bound': '99'}",0.168434,0.168434,"{'65+': 0.285042, '55-64': 0.27561899999999995..."


In [ ]:
#https://datascience.stackexchange.com/questions/9616/how-to-create-us-state-choropleth-map
#Convert the demographic data into a heat-map


In [ ]:
data.loc[[0]]["delivery_by_region"]


0    [{'percentage': '1', 'region': 'Wisconsin'}]
Name: delivery_by_region, dtype: object